In [1]:
import csv
import random
import datetime
import time
import os

File paths


In [2]:
questionsFile = "questions.csv"
scoresFile = "scores.csv"

Loading / Finding questions from CSV file
bold text

In [3]:
def loadQuestions():
  questions = []

  if not os.path.exists(questionsFile):
        with open(questionsFile, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['id', 'text', 'option_a', 'option_b', 'option_c', 'option_d', 'correct_answer', 'marks'])
        return questions

  with open(questionsFile, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
      questions.append({
          'id' : row['id'],
          'text': row['text'],
          'options': {
              'A': row['option_a'],
              'B': row['option_b'],
              'C': row['option_c'],
              'D': row['option_d'],
          },
          'correct_answer' : row['correct_answer'],
          'marks': int(row['marks'])
      })
  return questions


**Save the questions**

In [4]:
def saveQuestions(questions):
  with open(questionsFile, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'text', 'option_a', 'option_b', 'option_c', 'option_d', 'correct_answer', 'marks'])
    for question in questions:
      writer.writerow([
          question['id'],
          question['text'],
          question['options']['A'],
          question['options']['B'],
          question['options']['C'],
          question['options']['D'],
          question['correct_answer'],
          question['marks']
      ])

**Function for add questions**


In [5]:
def addQuestion(questions):
  question = input("Enter your questions here: ")
  optn= {
      'A': input("Enter your option A: "),
      'B': input("Enter your option B: "),
      'C': input("Enter your option C: "),
      'D': input("Enter your option D: "),
  }
  correctAnswer = input("Enter correct answer(A/B/C/D): ").upper()
  while correctAnswer not in ['A', 'B', 'C', 'D']:
    print("Invalid answer! Answer should be A, B, C or D")
    correctAnswer = input("Enter correct answer(A/B/C/D): ").upper()
  while True:
    try:
      marks = int(input("Enter marks for the question: "))
      break
    except ValueError:
      print("Invalid marks! Marks should be a number")
  new_question = {
      'id': str(len(questions) + 1),
      'text': question,
      'options': optn,
      'correct_answer': correctAnswer,
      'marks': marks
  }
  questions.append(new_question)
  saveQuestions(questions)
  print("Question added successfully")


**Function for update questions**

In [6]:
def updateQuestion(questions):
  if not questions:
    print("No questions found")
    return

  questionId = input("Enter question ID to update")
  question = None

  for q in questions:
    if q['id']== questionId:
      question = q
      break

    if question is None:
      print("Question not found")
      return

  text = input("Enter updated version of question or press Enter to skip")

  if text:
    question['text'] = text

  if input("Update options? (y/n): ").lower()=='y':
    for option in ['A', 'B', 'C', 'D']:
      text = input(f"Enter updated version of option {option} or press Enter to skip")
      if text:
        question['options'][option] = text

  correctAnswer = input("Enter correct answer or press Enter to skip").upper()

  if correctAnswer:
    while correctAnswer not in ['A', 'B', 'C', 'D']:
      print("Invalid answer!Answer should be A,B,C or D")
      correctAnswer = input("Enter correct answer(A/B/C/D)").upper()
    question['correct_answer'] = correctAnswer

  marks = input("Enter updated marks or press Enter to skip")

  if marks:
    while True:
      try:
        question['marks'] = int(marks)
        break
      except ValueError:
        print("Invalid marks!Marks should be a number")
        marks = input("Enter updated marks or press Enter to skip")

  saveQuestions(questions)
  print("Questions Updated successfully")




**Function for delete questions**

In [7]:
def deleteQuestion(questions):
  if not questions:
    print("No questions found")
    return

  id = input("Enter the ID of the question you want ot delete: ")
  for i, q in enumerate(questions):
    if q['id'] == id:
      del questions[i]
      saveQuestions(questions)
      print("Question deleted successfully")
      return
  print("Question not found")


**Function for display all the questions**

In [8]:
def displayQuestions(questions):
  if not questions:
    print("No questions found")
    return

  print("All the questions are:")
  print("=" *30)

  for q in questions:
    print(f"Question ID: {q['id']}")
    print(f"Question: {q['text']}")
    print("Options:")

    for option, text in q['options'].items():
      print(f"{option}: {text}")
    print(f"Correct Answer: {q['correct_answer']}")
    print(f"Marks: {q['marks']}")
    print("-" * 50)


**Fucntion for save scores**

In [9]:

def saveScore(participant_name, score):
    """Save quiz score to CSV file."""
    with open(scoresFile, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([
            participant_name,
            datetime.datetime.now().strftime("%Y-%m-%d "),
            score
        ])

**Function for display scores**

In [10]:
def displayScores():
    """Display all past quiz scores."""
    if not os.path.exists(scoresFile):
        print("No scores available yet!")
        return

    print("\nPast Scores:")
    print("=" * 50)
    with open(scoresFile, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            print(f"Participant: {row[0]}, Date: {row[1]}, Score: {row[2]}")


**Function for make Quiz**

In [11]:
def makeQuiz(questions):

    if not questions:
        print("No questions available in the quiz!")
        return

    participant_name = input("Enter your name: ")
    numberOfQuestions = int(input("Enter number of questions: "))
    time_limit = input("Enter time limit in seconds (press Enter for no limit): ")
    time_limit = int(time_limit) if time_limit else None

    quiz_questions = random.sample(questions, min(numberOfQuestions, len(questions)))
    score = 0
    start_time = time.time()

    for question in quiz_questions:
        if time_limit and (time.time() - start_time) > time_limit:
            print("\nTime's up!")
            break

        print(f"\nQuestion: {question['text']}")
        for option, text in question['options'].items():
            print(f"{option}: {text}")

        while True:
            answer = input("\nYour answer (A/B/C/D): ").upper()
            if answer in ['A', 'B', 'C', 'D']:
                break
            print("Invalid input! Please enter A, B, C, or D.")

        if answer == question['correct_answer']:
            score += question['marks']
            print("Correct!")
        else:
            print(f"Incorrect. The correct answer was {question['correct_answer']}")

    # Calculate and display results
    total_possible = sum(q['marks'] for q in quiz_questions)
    percentage = (score / total_possible) * 100
    grade = 'A' if percentage >= 90 else 'B' if percentage >= 80 else 'C' if percentage >= 70 else 'D' if percentage >= 60 else 'F'

    print("\nQuiz Summary")
    print("=" * 30)
    print(f"Participant: {participant_name}")
    print(f"Score: {score}/{total_possible}")
    print(f"Percentage: {percentage:.2f}%")
    print(f"Grade: {grade}")

    saveScore(participant_name, score)


Take Decision what you wanna do
**bold text**

In [12]:
def main():
    while True:
        print("\nQuiz Management System")
        print("1. Add Question")
        print("2. Update Question")
        print("3. Delete Question")
        print("4. Take Quiz")
        print("5. View Scores")
        print("6. View All Questions")
        print("7. Exit")

        choice = input("Enter your choice (1-7): ")
        questions = loadQuestions()

        if choice == '1':
            addQuestion(questions)
        elif choice == '2':
            updateQuestion(questions)
        elif choice == '3':
            deleteQuestion(questions)
        elif choice == '4':
            makeQuiz(questions)
        elif choice == '5':
            displayScores()
        elif choice == '6':
            displayQuestions(questions)
        elif choice == '7':
            print("Thank you for using the Quiz Management System!")
            break
        else:
            print("Invalid choice! Please try again.")



## Main

In [ ]:
if __name__ == "__main__":
    main()


Quiz Management System
1. Add Question
2. Update Question
3. Delete Question
4. Take Quiz
5. View Scores
6. View All Questions
7. Exit
Enter your choice (1-7): 6
All the questions are:
Question ID: 1
Question: What is the capital of France?
Options:
A: Paris
B: Lyon
C: Marseille
D: Nice
Correct Answer: A
Marks: 5
--------------------------------------------------
Question ID: 2
Question: Which planet is known as the Red Planet?
Options:
A: Venus
B: Mars
C: Jupiter
D: Saturn
Correct Answer: B
Marks: 5
--------------------------------------------------
Question ID: 3
Question: Who wrote 'Hamlet'?
Options:
A: Charles Dickens
B: William Shakespeare
C: Mark Twain
D: Jane Austen
Correct Answer: B
Marks: 10
--------------------------------------------------
Question ID: 4
Question: What is 5 + 7?
Options:
A: 11
B: 12
C: 13
D: 14
Correct Answer: B
Marks: 5
--------------------------------------------------
Question ID: 5
Question: Which element has the atomic number 1?
Options:
A: Helium
B: O